In [5]:
import os
import pandas as pd
import torch as tc
import numpy as np
from tqdm import tqdm

from star.star import STAR

from torch.autograd import Variable
from pytorch3d.loss import point_mesh_face_distance
from pytorch3d.structures import Meshes, Pointclouds
from curve_utils import CurveUtils
from mesh_manipulation import load_mesh, load_template, save_obj

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

# functions

In [6]:
def zero_poses(poses):
    poses = poses.clone().detach()
    poses[:,12:17] = 0
    poses[:,54:65] = 0
    poses = Variable(poses, requires_grad=True)
    return poses

# fiting

In [16]:
status_data = pd.DataFrame(columns=["gender", 'iteration', 'loss', 'elapsed_time'])

for gender in ['male', 'female']:

    # loading bodies
    files_path = f"./data/MOVE4D/{gender}/"
    files = os.listdir(files_path)
    files.sort()

    all_vertices = []
    all_faces = []
    files_iterator = tqdm(files, desc=f"loading {gender} bodies", position=0)
    for file in files_iterator:
        body_vertices = load_mesh(files_path + file, device)
        body_vertices *= 0.001
        body_vertices -= body_vertices.mean(axis=0)
        all_vertices.append(body_vertices)

    body = Pointclouds(points=all_vertices)[0]

#     nbetas_iterator = tqdm(range(10, 300, 1), desc=f"fitting {gender} with nbetas")
    for nbetas in range(10, 300, 1):

        all_betas = []
        all_poses = []
        all_trans = []

        poses = tc.FloatTensor(tc.zeros((1, 72))).to(device)
        poses = Variable(poses, requires_grad=True)
        betas = tc.FloatTensor(tc.zeros((1, nbetas))).to(device)
        betas = Variable(betas, requires_grad=True)
        trans = tc.FloatTensor(tc.zeros((1, 3))).to(device)
        trans = Variable(trans, requires_grad=True)
        star = STAR(gender=gender, num_betas=nbetas)
        star_faces = star.faces[None,...].to(device)

        # training model
        learning_rate = 0.02
        epochs = 5000
        optimizer = tc.optim.Adam([trans, betas, poses], lr=learning_rate)
        epochs_iterator = tqdm(range(epochs), desc=f"optimizing - loss: inf")
        for it in epochs_iterator:

            if it == 20:
                poses = zero_poses(poses)
                
            d = star(betas=betas, pose=poses, trans=trans)
            optimizer.zero_grad()
            star_meshes = Meshes(verts=d, faces=star_faces)
            loss = point_mesh_face_distance(star_meshes, body)
            loss.backward(retain_graph=True)
            epochs_iterator.set_description(f"optimizing - loss: {loss}")
            optimizer.step()
            elapsed_time = epochs_iterator.format_dict['elapsed']
            if loss < 2e-5:
                break
            if elapsed_time > 600:
                break
        status_data.loc[len(status_data.index)] = [gender, it, loss.detach().cpu().numpy(), elapsed_time]
        
status_data.to_csv("status_data.csv")
  

optimizing - loss: 0.00016335120017174631:  71%|████████████████████████████████████████                | 3573/5000 [10:00<03:59,  5.95it/s]


optimizing - loss: 0.00010078663763124496:  72%|████████████████████████████████████████                | 3575/5000 [10:00<03:59,  5.96it/s]


optimizing - loss: 6.403037696145475e-05:  71%|████████████████████████████████████████▋                | 3571/5000 [10:00<04:00,  5.95it/s]


optimizing - loss: 5.27892516402062e-05:  72%|█████████████████████████████████████████▍                | 3575/5000 [10:00<03:59,  5.96it/s]


optimizing - loss: 4.6792229113634676e-05:  71%|███████████████████████████████████████▉                | 3571/5000 [10:00<04:00,  5.95it/s]


...

optimizing - loss: 7.01924946042709e-05:  71%|█████████████████████████████████████████                 | 3543/5000 [10:00<04:06,  5.90it/s]


optimizing - loss: 5.2034316468052566e-05:  73%|████████████████████████████████████████▊               | 3642/5000 [10:00<03:43,  6.07it/s]


optimizing - loss: 3.967862721765414e-05:  73%|█████████████████████████████████████████▌               | 3647/5000 [10:00<03:42,  6.08it/s]


optimizing - loss: 3.431987715885043e-05:  73%|█████████████████████████████████████████▌               | 3644/5000 [10:00<03:43,  6.07it/s]


optimizing - loss: 3.476258279988542e-05:  73%|█████████████████████████████████████████▌               | 3645/5000 [10:00<03:43,  6.07it/s]
